# Data Integration with CHAKRA

## Import CHAKRA library

In [1]:
using Chakra

┌ Info: Precompiling Chakra [23337c43-347c-4169-b193-f31a84c326bf]
└ @ Base loading.jl:1423


## Implement an interface for a data source S1

The interface must define the core CHAKRA types and operations.

In [2]:
module S1
    using Chakra
    Chakra.@Reference Int []
    
    # DEFINE ATTRIBUTES

    Chakra.@Attribute(:A,Int)
    Chakra.@Attribute(:B,String)

    o = Chakra.seta(Att(:A),10,c(Id[]))

    data = Chakra.ins(Id(1),o,emp(Hierarchy))

end

Main.S1

In [3]:
typeof(S1.data)

Main.S1.Hierarchy

In [4]:
dom(S1.data)

1-element Vector{Main.S1.Id}:
 Main.S1.Id(1)

In [5]:
geta(:A,S1.Id(1),S1.data)

10

## Implement a second data source interface S2

In [9]:
module S2
    using Chakra
    Chakra.@Reference Int []

    data = Chakra.ins(Id(2),c(Id[]),emp(Hierarchy))

end

Main.S2

In [10]:
dom(S2.data)

1-element Vector{Main.S2.Id}:
 Main.S2.Id(2)

## Link data sources S1 and S2 using a third data source S3

In [11]:
module S3
    using Chakra
    using Main.S1, Main.S2

    Chakra.@Reference Int [S1,S2]
    
    Chakra.fnd(x::S1.Id,h::Hierarchy) = fnd(x,S1.data)
    Chakra.fnd(x::S2.Id,h::Hierarchy) = fnd(x,S2.data)
    Chakra.dom(h::Hierarchy) = vcat(collect(keys(h.constituents)),dom(S1.data),dom(S2.data))

    data = Chakra.ins(Id(3),c(DOMAIN[S1.Id(1),S2.Id(2)]),emp(Hierarchy))

end

Main.S3

In [12]:
dom(S3.data)

3-element Vector{Any}:
 Main.S3.Id(3)
 Main.S1.Id(1)
 Main.S2.Id(2)

In [13]:
o = fnd(S3.Id(3),S3.data)

Main.S3.Constituent(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Union{Main.S1.Id, Main.S2.Id, Main.S3.Id}[Main.S1.Id(1), Main.S2.Id(2)])

In [14]:
l = pts(o)

2-element Vector{Union{Main.S1.Id, Main.S2.Id, Main.S3.Id}}:
 Main.S1.Id(1)
 Main.S2.Id(2)

In [15]:
sequence(S3.Id(3),S3.data)

2-element Vector{Any}:
 Main.S1.Constituent(Dict{Symbol, Any}(:A => 10), Dict{Symbol, Any}(), Main.S1.Id[])
 Main.S2.Constituent(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Main.S2.Id[])

In [16]:
sequence(l,S3.data)

2-element Vector{Any}:
 Main.S1.Constituent(Dict{Symbol, Any}(:A => 10), Dict{Symbol, Any}(), Main.S1.Id[])
 Main.S2.Constituent(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Main.S2.Id[])